# An Introduction to Motion Kinetics Model
_**The Modjoul Motion Kinetics Model provides information about ergonomic behavior — duration and counts of walking, bending and twisting like activities based upon various sensor values.**_

1. [Introduction](#1.-Introduction)
2. [Preparing CSV](#2.-Preparing-CSV)
3. [Method - 1 Using Endpoint](#3.-Using-Endpoint)<br>
    3.1. [Creating model](#3.1.-Creating-Model)<br>
    3.2. [Creating endpoint config](#3.2.-Creating-Endpoint-Config)<br>
    3.3. [Creating endpoint](#3.3.-Creating-Endpoint)<br>
    3.2. [Invoking endpoint](#3.4.-Invoking-Endpoint)<br>
    3.2. [Deleting the Endpoint - Optional](#3.5.-Deleting-the-Endpoint---Optional)
4. [Method - 2 Using batch transform job](#4.-Using-Batch-Transform-Job)
5. [Model Output](#5.-Model-Output)


## 1. Introduction
Motion is the number one cause for injuries in the workplace as well as the structural driver for athletic activities. Modjoul Motion model identifies 4 major activities and corresponding metrics. This model detects not only walking steps but also bends, twists and stationary time. It accepts input in CSV format and outputs data in JSON format.<br>
Bending – Count and duration at different bending angle buckets with/without acceleration and twist<br>
Twisting – Count and duration for low and high angle twists with/without acceleration<br>
Walking – Steps, duration, distance and speed<br>
Stationary work – Duration<br>

## 2. Preparing CSV
Prepare a csv file that should and must contain at least the following sensor values as columns:<br>
**altimeter, accelerometer (X,Y,Z-axis values), magnetometer (X,Y,Z-axis values), gyroscope (X,Y,Z-axis values)**. The X, Y, Z - axis are aligned such that +X points to left of the body, +Y points towards above the body and +Z points to the front of the body. The model is a best fit if the sensors placement is at the waist region (similar to the postion of belt buckle).<br><br>
**GPS** sensor values are optional but can be provided in order to improve the accuracy of the model. GPS values should contain two columns **gpsSpeed and gpsCourse**, if they are available.
<font color="red">_**Each data point is considered at milisecond level i.e, 10 rows = 1 second worth data**_</font> <br><br>
The CSV file should include header and the columnNames should be as below : <br>
_**altimeter accelX accelY accelZ magnetoX magnetoY magnetoZ gyroX gyroY gyroZ gpsSpeed gpsCourse**_
(last two columns are optional)

**Note:** The request body size for the endpoint should not exceed 7 MB.

In [1]:
import pandas as pd
df = pd.read_csv('https://s3.amazonaws.com/sagemaker-sample-datasets/IndoorDriving/InputData/sampleInput.csv')
df.to_csv("sample.csv",index=False)
df.head(3)

,timestamp,altimeter,gyroX,gyroY,gyroZ,accelX,accelY,accelZ,magnetoX,magnetoY,magnetoZ,gpsSpeed,gpsCourse
0,2017-09-08T18:08:10.000Z,201.07,0.31,-0.55,1.59,-0.32,-9.35,3.88,5,-5,130,0.0,0.0
1,2017-09-08T18:08:10.001Z,149.10,0.49,-0.49,1.46,-0.38,-9.30,3.89,3,58,71,0.0,0.0
2,2017-09-08T18:08:10.002Z,148.75,0.06,-0.49,1.22,-0.41,-9.35,3.88,5,60,73,0.0,0.0


## 3. Using Endpoint

The model can be directly accessed via the console provided by AWS. However, for a more customized process one can access the model using the below code as well.

### 3.1. Creating Model
To create a model, import boto3, sagemaker and get the arn of the model package

In [ ]:
import boto3
import sagemaker
role = sagemaker.get_execution_role()
smmp = boto3.client('sagemakermp')
modelName='<Input Model Name>'' 
modelArn = 'Paste Model ARN>'
createHeatIndexResponse = smmp.create_model(ModelName='Model-From-modjoul-motion',\
                             Containers=[{'ModelPackageName': modelArn}],\
                             ExecutionRoleArn=role,\
                             EnableNetworkIsolation=True )

### 3.2. Creating Endpoint Config


In [ ]:
configName='<Input Configuration Name>'
instanceType = '<Input Instance Type>'
createHeatIndexEndpointConfig = smmp.create_endpoint_config(EndpointConfigName = configName, ProductionVariants = [{'InstanceType':instanceType, 'InitialInstanceCount':1, 'ModelName':modelName, 'VariantName':'xyz'}])

### 3.3. Creating Endpoint

In [ ]:
endpointName = '<Input Endpoint Name>'
createHeatIndexEndpoint = smmp.create_endpoint(EndpointName = endpointName, EndpointConfigName = configName)

### 3.4. Invoking Endpoint

In [5]:

runtime = boto3.Session().client('runtime.sagemaker')

#Reading Input Data 
with open('sample.csv','rb') as f:
    payload = f.read()

response = runtime.invoke_endpoint(EndpointName = endpointName, ContentType = 'text/csv', Body = payload)
result = response['Body'].read().decode()

#Writing Output Data 
import json
with open('sampleOutput.json','w') as f:
    f.write(json.dumps(json.loads(result)))

In [11]:
response = runtime.invoke_endpoint(EndpointName = endpointName, ContentType = 'text/csv', Body = payload)

### 3.5. Deleting the Endpoint - Optional

If you're ready to be done with this notebook, please run the delete_endpoint line in the cell below.  This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [ ]:
sagemaker.Session().delete_endpoint(endpointName)

## 4. Using Batch Transform Job

Refer the below link for how to use batch transform job for getting inferences from a model
[sagemaker batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html#ex1-batch-transform-console)

In [ ]:
import boto3
import sagemaker

inputLocation = '<S3 location for Input Data>'
outputLocation = '<S3 location for Output Data>'

jobName = '<Input Job Name>'
modelName = '<Input Model Name>'
instanceType = '<Input Instance Type>

# Initialize the transformer object
transformer =sagemaker.transformer.Transformer(base_transform_job_name = jobName, model_name = modelName, instance_count=1, instance_type = instanceType, output_path = outputLocation)

# To start a transform job:
transformer.transform(inputLocation, content_type='text/csv', split_type='None')

# Then wait until transform job is completed
transformer.wait() 


## 5. Model Output

In [7]:
import json
with open("./sampleOutput.json","r") as f:
    sampleResponse =json.loads(f.read())
sampleResponse

{'others': {'othersDuration': 0.0},
 'totalDuration': {'totalDuration': 15.0},
 'scores': {'scoreLumbarStatic': 10.8,
  'scoreLumbarDynamic': 4,
  'scoreMET': 2.3},
 'laying': {'layingDuration': 0.0},
 'bending': {'bendAccTwist61_70Count': 0,
  'bendOnly41_50Count': 0,
  'bendAcc21_30Count': 0,
  'bend41_50Duration': 0.0,
  'bendTotalDuration': 0.0,
  'bendAcc51_60Count': 0,
  'bendAcc61_70Count': 0,
  'bendAccTwist31_40Count': 0,
  'bend31_40Count': 0,
  'bendTwistCount': 0,
  'bendAccCount': 0,
  'bendAccTwist41_50Count': 0,
  'bendTwist71_Count': 0,
  'bend21_30Count': 0,
  'bendTwist61_70Count': 0,
  'bendTwist51_60Count': 0,
  'bend21_30Duration': 0.0,
  'bendOnly51_60Count': 0,
  'bend61_70Duration': 0.0,
  'bendTwist31_40Count': 0,
  'bendOnly71_Count': 0,
  'bend71_Count': 0,
  'bendAccTwistCount': 0,
  'bendTwist21_30Count': 0,
  'bendAccTwist21_30Count': 0,
  'bend41_50Count': 0,
  'bendOnly61_70Count': 0,
  'bend61_70Count': 0,
  'bendTwist41_50Count': 0,
  'bendOnlyCount': 